In [ ]:
from sklearn.model_selection import train_test_split

# Manually split the dataframe into training and validation sets
train_df, val_df = train_test_split(merged_data, test_size=0.2, random_state=42, shuffle=True)

# Extract and scale metadata for training and validation sets
X_train_metadata = scaler.fit_transform(train_df[metadata_features].values)
X_val_metadata = scaler.transform(val_df[metadata_features].values)


datagen = ImageDataGenerator(rescale=1./255)
# Create ImageDataGenerator instances for training and validation
train_image_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='train-image/image/',
    x_col='image',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_image_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='train-image/image/',
    x_col='image',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Update the custom generator to use the split metadata
def custom_generator(image_gen, metadata):
    while True:
        image_batch, label_batch = next(image_gen)
        indices = image_gen.index_array
        metadata_batch = metadata[indices]
        yield {'image_input': image_batch, 'metadata_input': metadata_batch}, label_batch

# Training the model using the updated generator
history = model.fit(
    custom_generator(train_image_generator, X_train_metadata),
    steps_per_epoch=len(train_image_generator),
    epochs=10,
    validation_data=custom_generator(validation_image_generator, X_val_metadata),
    validation_steps=len(validation_image_generator)
)
